# 解析装备页面

包含基础装备，传奇装备，英雄追忆以及装备词条。

词条：
* 基础词条
* 侵蚀词条
* 解梦词条

In [3]:
import json
from bs4 import BeautifulSoup

In [4]:
import os
import pandas as pd

In [5]:
import re

In [6]:
def save_json(obj, fpath):
    if not os.path.isdir('../data/json/'):
        os.mkdir('../data/json/')
    with open(os.path.join('../data/json/',fpath),'w',encoding='utf-8') as file:
        json.dump(obj,file,ensure_ascii=False)

## 1. 英雄追忆页面

包括主要天赋，小型、中型天赋、传奇中型天赋和新神天赋

https://tlidb.com/cn/Talent

In [106]:
memory_key_map = {
    'Memory_of_Origin':'Memory_of_Origin',
    'Memory_of_Discipline':'Memory_of_Discipline',
    'Memory_of_Progress':'Memory_of_Progress',
}

In [120]:
memory_attrs_key_map = {
    'base_attrs':'基础属性',
    'fix_attrs':'固有词缀'
}

In [176]:
def unpack_nested(nodes,):
    nodes += ['']
    for hdx in range(0,len(nodes)-1):
        nodes[hdx] = nodes[hdx].replace(nodes[hdx+1],'')
    nodes.pop()
    return nodes

In [181]:
def parse_nested_head(node):
    headers = [i.text.strip() for i in node.find('thead').find('tr').find_all('th')]
    headers = unpack_nested(headers)
    return headers

In [183]:
def parse_nested_body(node):
    bodies = unpack_nested([i.text.strip() for i in BeautifulSoup(node,'html.parser').find_all('td')])
    return bodies

In [188]:
memory_data = {}
for memo_name in memory_key_map:
    
    html_code = open(f'../data/site/cn/{memo_name}.html').read()
    
    soup = BeautifulSoup(html_code, 'html.parser')
    
    memo_node = dict()
    
    for attr_key in memory_attrs_key_map:
        print(memo_name, attr_key, memory_attrs_key_map[attr_key])
        
        soup_tb = soup.find('div',id=memory_attrs_key_map[attr_key])
        
        # 获取表头
        headers = parse_nested_head(soup_tb)
        
        bodies = [i for i in str(soup_tb.find('tbody').find('tr')).replace('</tr>','').replace('</td>','').replace('<tr>','\n').split('\n') 
                              if i!='']
        
        # 初始化结果列表
        data = []
        
        # 遍历每一行数据
        for row in bodies:
            cells = parse_nested_body(row)
            row_data = {k:v for k,v in zip(headers,cells)}
            
            # 将字典添加到结果列表中
            data.append(row_data)
        
        memo_node[attr_key] = data

    memory_data[memo_name] = memo_node

Memory_of_Origin base_attrs 基础属性
Memory_of_Origin fix_attrs 固有词缀
Memory_of_Discipline base_attrs 基础属性
Memory_of_Discipline fix_attrs 固有词缀
Memory_of_Progress base_attrs 基础属性
Memory_of_Progress fix_attrs 固有词缀


In [189]:
save_json(memory_data, 'memory.json')

## 2. 装备界面

In [190]:
base_equip_nodes = {}

In [194]:
# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(open('../data/site/cn/Craft.html'), 'html.parser')

# 找到id为MainTalentNode的div标签
profession_node = soup.find(name='div', attrs={"id":talent_key_map["Profession"]})

for col_node in profession_node.find_all(name='div',attrs={"class":"col"}):
    
    href = col_node.find('a').attrs['href']
    text_name = col_node.find(name='a').text
    main_attrs = re.findall(r'主要属性：(\w+)',str(col_node))
    main_tag = col_node.find('richtext').contents[0].split('、') if col_node.find('richtext').contents else []
    
    talent_nodes[href] = {'talent_name':text_name,
                         'talent_main_attribute':main_attrs,
                         'talent_tags':main_tag,
                         'main_talent_lst':[],
                         'mini_talent_lst':[],
                         'mid_talent_lst':[],
                          'legend_mid_talent_lst':[]
                         }

In [195]:
talent_nodes

{'God_of_Might': {'talent_name': '巨力之神',
  'talent_main_attribute': ['力量'],
  'talent_tags': ['攻击', '火焰'],
  'main_talent_lst': [],
  'mini_talent_lst': [],
  'mid_talent_lst': [],
  'legend_mid_talent_lst': []},
 'Goddess_of_Hunting': {'talent_name': '狩猎之神',
  'talent_main_attribute': ['敏捷'],
  'talent_tags': ['速度', '闪电'],
  'main_talent_lst': [],
  'mini_talent_lst': [],
  'mid_talent_lst': [],
  'legend_mid_talent_lst': []},
 'Goddess_of_Knowledge': {'talent_name': '知识之神',
  'talent_main_attribute': ['智慧'],
  'talent_tags': ['法术', '冰冷'],
  'main_talent_lst': [],
  'mini_talent_lst': [],
  'mid_talent_lst': [],
  'legend_mid_talent_lst': []},
 'God_of_War': {'talent_name': '征战之神',
  'talent_main_attribute': ['力量'],
  'talent_tags': ['几率', '物理'],
  'main_talent_lst': [],
  'mini_talent_lst': [],
  'mid_talent_lst': [],
  'legend_mid_talent_lst': []},
 'Goddess_of_Deception': {'talent_name': '欺诈之神',
  'talent_main_attribute': ['敏捷'],
  'talent_tags': ['持续', '腐蚀'],
  'main_talent_lst': 

### 1.2 解析主要天赋

In [196]:
# 找到id为MainTalentNode的div标签
main_talent_node = soup.find(name='div', attrs={"id":talent_key_map["MainTalentNode"]})

for col_node in main_talent_node.find_all(name='div',attrs={"class":"col"}):
    main_talent = {
                'talent_name':'',
        'talent_id':'',
              'talent_description':'',
                'talent_type':'main'
                  }
    
    tid = col_node.find('img').attrs['src'].split('\\')[1].removesuffix('.webp')
    
    name = col_node.find_all('span')[0].text
    talent_key = col_node.find('a').attrs['href']
    desc = col_node.text
    main_talent['talent_name'] = name
    main_talent['talent_description'] = desc
    main_talent['talent_id'] = tid
    
    talent_nodes[talent_key]['main_talent_lst'].append(main_talent)

In [198]:
talent_nodes

{'God_of_Might': {'talent_name': '巨力之神',
  'talent_main_attribute': ['力量'],
  'talent_tags': ['攻击', '火焰'],
  'main_talent_lst': [{'talent_name': '淘汰',
    'talent_id': 'UI_CoreTalentIcon_taotai_Icon_128',
    'talent_description': '淘汰巨力之神攻击击中时，淘汰生命值低于 18% 的敌人',
    'talent_type': 'main'},
   {'talent_name': '燃尽',
    'talent_id': 'UI_CoreTalentIcon_ranjin_Icon_128',
    'talent_description': '燃尽巨力之神额外 +24% 火焰伤害重创敌人时，使敌人 +20% 火焰抗性，持续 1 秒',
    'talent_type': 'main'},
   {'talent_name': '坚韧',
    'talent_id': 'UI_CoreTalentIcon_jianren_Icon_128',
    'talent_description': '坚韧巨力之神击中敌人时，+100% 几率获得 1 层坚韧祝福+1 坚韧祝福层数上限',
    'talent_type': 'main'},
   {'talent_name': '势大力沉',
    'talent_id': 'UI_CoreTalentIcon_shidalichen_Icon_128',
    'talent_description': '势大力沉巨力之神-10% 攻击速度额外 +30% 攻击伤害额外 +30% 攻击造成的异常伤害',
    'talent_type': 'main'},
   {'talent_name': '火上浇油',
    'talent_id': 'UI_CoreTalentIcon_huoshangjiaoyou_Icon_128',
    'talent_description': '火上浇油巨力之神伤害无视敌人火焰抗性',
    'talent_type': 'ma

### 1.3 解析中小型天赋

In [199]:
talent_type_map = {'小型天赋':'mini_talent_lst',
                   '中型天赋':'mid_talent_lst',
                   '传奇中型天赋':'legend_mid_talent_lst'
                  }

In [200]:
# 找到id为MainTalentNode的div标签
talent_node = soup.find(name='div', attrs={"id":talent_key_map["Talent"]})

for col_node in talent_node.find_all(name='div',attrs={"class":"col"}):
    talent_node = {'talent_id':'',
                   'talent_name':'',
                   'talent_type':'',
                  'talent_description':''}
    tid = col_node.find('img').attrs['src'].split('\\')[1].removesuffix('.webp')

    href = col_node.find('a').attrs['href']

    talent_type = col_node.find_all('span')[0].text

    desc = ''.join([i.text for i in col_node.find('div',class_="flex-grow-1 mx-2 my-1")][1:])

    insert_key = talent_type_map[talent_type]
    
    talent_node['talent_id'] = tid
    talent_node['talent_description'] = desc
    talent_node['talent_type'] = insert_key.removesuffix('_talent_lst')

    talent_nodes[href][insert_key].append(talent_node)

In [201]:
talent_nodes

{'God_of_Might': {'talent_name': '巨力之神',
  'talent_main_attribute': ['力量'],
  'talent_tags': ['攻击', '火焰'],
  'main_talent_lst': [{'talent_name': '淘汰',
    'talent_id': 'UI_CoreTalentIcon_taotai_Icon_128',
    'talent_description': '淘汰巨力之神攻击击中时，淘汰生命值低于 18% 的敌人',
    'talent_type': 'main'},
   {'talent_name': '燃尽',
    'talent_id': 'UI_CoreTalentIcon_ranjin_Icon_128',
    'talent_description': '燃尽巨力之神额外 +24% 火焰伤害重创敌人时，使敌人 +20% 火焰抗性，持续 1 秒',
    'talent_type': 'main'},
   {'talent_name': '坚韧',
    'talent_id': 'UI_CoreTalentIcon_jianren_Icon_128',
    'talent_description': '坚韧巨力之神击中敌人时，+100% 几率获得 1 层坚韧祝福+1 坚韧祝福层数上限',
    'talent_type': 'main'},
   {'talent_name': '势大力沉',
    'talent_id': 'UI_CoreTalentIcon_shidalichen_Icon_128',
    'talent_description': '势大力沉巨力之神-10% 攻击速度额外 +30% 攻击伤害额外 +30% 攻击造成的异常伤害',
    'talent_type': 'main'},
   {'talent_name': '火上浇油',
    'talent_id': 'UI_CoreTalentIcon_huoshangjiaoyou_Icon_128',
    'talent_description': '火上浇油巨力之神伤害无视敌人火焰抗性',
    'talent_type': 'ma

## 2. 保存数据

In [204]:
save_json(talent_nodes,'talent.json')